In [1]:
import glob
import json
import ipaddress

In [2]:
def read_hosts(path):
    nws = set()
    for h in glob.glob(path):
        with open(h) as fp:
            content = json.load(fp)
            nws.add(ipaddress.ip_network(content['hostInterfaces']['eth0']['prefix'], strict=False))
    return nws

In [3]:
def find_in_lines(needle, search):
    diff_specs = []
    for i, l in enumerate(search.splitlines() if type(search) is str else search):
        # print(needle.__str__())
        if needle.__str__() in l:
            diff_specs.append(l)
            # print(f"Found {needle} in line {i}:", l)
    return diff_specs

In [4]:
from pybatfish.client.session import Session
def find_config_prefixes(path):
  bf = Session("127.0.0.1", 9997, 9996)
  bf.set_network("test")
  bf.init_snapshot(path, name="test", overwrite=True)

  # Get the prefixes from the config
  # interfaces = bf.q.interfaceProperties().answer().frame()
  # print(dir(bf.q))
  dat = set()
  interfaces = bf.q.interfaceProperties().answer().frame()

  for pfxs in interfaces['All_Prefixes']:
    # print(pfxs)
    for pfx in pfxs:
      dat.add(ipaddress.ip_network(pfx, strict=False))

  return dat
  # print(interfaces)
  # print(path)
  

In [12]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/fattree08-ospf"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
print("network_name", "specs#","kept#","false_positive#","false_negative#","#false_negative_fakehost","#false_positive_fakehost","#false_negative_fakelink","#false_positive_fakelink","#false_negative_fake_dsts","#false_positive_fake_dsts","#hosts", sep='\t')
orig_config_prefixes = set()
orig_host_prefixes = set()

for nw in networks:
    nw_name = nw.split("/")[-1]
    find_config_prefixes(nw)
    # if "strawman" in nw_name:
    #     continue
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
        orig_config_prefixes = find_config_prefixes(nw)
        # print(orig_host_prefixes)
        # print(orig_config_prefixes)
    nw_host_pfxs = read_hosts(nw + "/hosts/host*.json")
    nw_config_prefixes = find_config_prefixes(nw)
    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv", 'r') as fp:
        nw_content = fp.read().__str__()
    with open(NETWORKS_DIR + f"/origin/policies.csv", 'r') as fp:
        origin_content = fp.read().__str__()
    
    origin_policies = set(origin_content.splitlines()[1:])
    nw_policies = set(nw_content.splitlines()[1:])
    fpos = nw_policies - origin_policies
    fneg = origin_policies - nw_policies
    kept = origin_policies & nw_policies

    # fpos and fneg introduced by fake hosts
    fneg_host_count = 0
    fpos_host_count = 0
    for pfx in nw_host_pfxs - orig_host_prefixes:
        diff_specs = find_in_lines(pfx, list(fneg))
        fneg_host_count += len(diff_specs)
        diff_specs = find_in_lines(pfx, list(fpos))
        # print(diff_specs)
        fpos_host_count += len(diff_specs)

    # fpos and fneg introduced by fake links
    fneg_link_count = 0
    fpos_link_count = 0
    for pfx in nw_config_prefixes - orig_config_prefixes:
        diff_specs = find_in_lines(pfx, list(fneg))
        fneg_link_count += len(diff_specs)
        diff_specs = find_in_lines(pfx, list(fpos))
        # print(diff_specs)
        fpos_link_count += len(diff_specs)
    
    fpos_fake_dst_count = fpos_link_count + fpos_host_count
    fneg_fake_dst_count = fneg_link_count + fneg_host_count

    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
        specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, specs_count, len(kept), len(fpos), len(fneg), fneg_host_count, fpos_host_count, fneg_link_count, fpos_link_count, fneg_fake_dst_count, fpos_fake_dst_count, len(nw_host_pfxs), sep="\t")
    # break

network_name	specs#	kept#	false_positive#	false_negative#	#false_negative_fakehost	#false_positive_fakehost	#false_negative_fakelink	#false_positive_fakelink	#false_negative_fake_dsts	#false_positive_fake_dsts	#hosts
origin	17322	17322	0	0	0	0	0	0	0	0	66
anonym-network-k4	28690	17157	11533	165	0	11476	0	11476	0	22952	132
strawman-1-6-4	46305	13155	33150	4167	0	30870	0	30870	0	61740	264
anonym-network-k6-4-w-noise	51660	17127	34533	195	0	34440	0	34440	0	68880	264
anonym-network-k6	28565	17004	11561	318	0	11426	0	11426	0	22852	132
anonym-network-k2-w-noise	28745	17178	11567	144	0	11498	0	11498	0	22996	132
anonym-network-k10	23505	11631	11874	5691	0	9402	0	9402	0	18804	132
strawman-2	25105	12924	12181	4398	0	10042	0	10042	0	20084	132
strawman-1	25030	12816	12214	4506	0	10012	0	10012	0	20024	132
anonym-network-k6-4	51588	17061	34527	261	0	34392	0	34392	0	68784	264


In [ ]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/bics-ospf"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
for nw in networks:
    nw_name = nw.split("/")[-1]
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
    host_pfxs_count = len(read_hosts(nw + "/hosts/host*.json"))
    # read diff
    with open(NETWORKS_DIR + f"/origin/not-in-{nw_name}.csv", 'r') as fp:
        diff_host_count = 0
        content = fp.read().__str__()
        for pfx in orig_host_prefixes:
            diff_specs = find_in_lines(pfx, content)
            diff_host_count += len(diff_specs)
    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
        specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, specs_count, len(content.splitlines()), diff_host_count, host_pfxs_count, sep="\t")
    # break